In [7]:
# Install Libries
!pip install requests
!pip install mysql.connector

In [16]:
# Import libries
import pandas as pd
import numpy as np
import requests
import mysql.connector

In [9]:
pd.set_option('display.max_columns',None)

# Countries Data

In [38]:
# importing the countries.
url = "https://api.worldbank.org/countries?format=json&per_page=300"

response = requests.get(url)
response.status_code

200

In [39]:
data = response.json()

In [40]:
countries = pd.json_normalize(data[1])
countries_clean = countries[countries['region.value']!='Aggregates'][['id','name','longitude','latitude','region.value','incomeLevel.value','lendingType.value']].rename(columns={"region.value":"region_value","incomeLevel.value":"incomeLevel_value","lendingType.value":"lendingType_value"})

In [41]:
countries_clean.head()

,id,name,longitude,latitude,region_value,incomeLevel_value,lendingType_value
0,ABW,Aruba,-70.0167,12.5167,Latin America & Caribbean,High income,Not classified
2,AFG,Afghanistan,69.1761,34.5228,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA
5,AGO,Angola,13.242,-8.81155,Sub-Saharan Africa,Lower middle income,IBRD
6,ALB,Albania,19.8172,41.3317,Europe & Central Asia,Upper middle income,IBRD
7,AND,Andorra,1.5218,42.5075,Europe & Central Asia,High income,Not classified


In [42]:
countries_clean.duplicated().sum()

0

In [43]:
countries_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 217 entries, 0 to 295
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 217 non-null    object
 1   name               217 non-null    object
 2   longitude          217 non-null    object
 3   latitude           217 non-null    object
 4   region_value       217 non-null    object
 5   incomeLevel_value  217 non-null    object
 6   lendingType_value  217 non-null    object
dtypes: object(7)
memory usage: 13.6+ KB


In [37]:
countries_clean[countries_clean['lendingType_value']==""]

,id,name,longitude,latitude,region_value,incomeLevel_value,lendingType_value


In [44]:
countries_clean['longitude'] = countries_clean['longitude'].replace("",-1.0)
countries_clean['latitude'] = countries_clean['longitude'].replace("",-1.0)

In [45]:
countries_clean['longitude'] = countries_clean['longitude'].astype('float')
countries_clean['latitude'] = countries_clean['longitude'].astype('float')


In [50]:
# save Data
countries_clean.to_csv('Countries_df.csv',index=False)

# DataBase Connection

In [17]:
conn = mysql.connector.connect(host='localhost',user='root',password='Jayshreekrishna9669@',database='worldbank',port=3306)
cursor = conn.cursor()

In [19]:
cursor.execute("""CREATE TABLE IF NOT EXISTS Countries(
                    country_id VARCHAR(10),
                    country_name VARCHAR(100),
                    longitude DOUBLE,
                    latitude DOUBLE,
                    region_name VARCHAR(100),
                    income_level VARCHAR(50),
                    lending_type VARCHAR(50)
)""")

In [46]:
for _,row in countries_clean.iterrows():
    cursor.execute("""
    INSERT INTO Countries(country_id,country_name,longitude,latitude,region_name,income_level,lending_type)
    VALUES(%s,%s,%s,%s,%s,%s,%s)
    """,tuple(row))

In [47]:
conn.commit()

In [48]:
cursor.execute("Drop Table Countries")